# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        
        # ~=Gain
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        #p_l = l_c/l_s
        #p_l**2 * l_s = l_c**2 / l_s
        #p_r = r_c/r_s
        #p_r**2 * r_s = r_c**2 / r_s 
        # деление на суммарное число элементов и сложение с константой не влияют на argmin
        return -np.sum(l_c**2 / l_s + r_c**2 / r_s, axis=1) # Ваш код
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        l_c = l_c.astype('float')
        r_c = r_c.astype('float')
        return -np.sum(l_c * np.log2(l_c/l_s + 1e-10) + r_c * np.log2(r_c/r_s + 1e-10), axis=1) # Ваш код 

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        #return - np.max(l_c/(l_s + r_s), axis=1) - np.max(r_c/(l_s + r_s), axis=1) # Ваш код
        return -(np.max(l_c, axis=1) + np.max(r_c, axis=1)) # Ваш код

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))] # Ваш код
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))] # Ваш код

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature) # Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        # разделение выборки по порогу
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # сортировка по признаку, соответствующая перестановка целевой y
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # число различных классов, представленных в y
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # элементы выборки за вычетом "запаса" на min_samples
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        # есть ли в упорядоченной выборке x последовательно расположенные элементы разных классов
        # их индексы в исходной, не урезанной, выборке
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        # если все метки в y принадлежат одному классу
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # число последовательных элементов одного класса в отсортированной выборке
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        # матрица соответствия индекса первого элемента класса в выборке и его номера
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        # матрица количества элементов в подпоследовательностях x, принадлежащих одному классу
        # это количество отмечено в элементе матрицы, аналогичном one_hot_code
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # возвращение первых min_samples элементов
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # число элементов каждого из представленных классов
        # с первого элемента выборки к текущему
        l_class_count = np.cumsum(class_increments, axis=0)
        # с последнего элемента выборки к текущему
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # выбор лучшего разбиения по impurity
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        # индекс разбиения по impurity
        left_el_id = l_sizes[idx][0]
        # значение impurity, значение threshold
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        
        if depth == self.max_depth or x.shape[0] <= self.min_samples_split:
            #leaf
            elems_in_classes = np.bincount(y, minlength=2)
            best_class = np.argmax(elems_in_classes)
            probs = elems_in_classes/x.shape[0]
            self.tree[node_id] = (self.LEAF_TYPE, best_class, probs)
            return
        
        if np.unique(y).shape[0] == 1:
            #leaf
            probs = np.zeros(2)
            probs[y[0]] = 1
            self.tree[node_id] = (self.LEAF_TYPE, y[0], probs)
            return
        
        if (np.max(np.bincount(y, minlength=2))).astype('float')/x.shape[0] > self.sufficient_share:
            #leaf
            elems_in_classes = np.bincount(y, minlength=2)
            best_class = np.argmax(elems_in_classes)
            probs = elems_in_classes/x.shape[0]
            self.tree[node_id] = (self.LEAF_TYPE, best_class, probs)
            return
        
        # выбор списка "рабочих" параметров по критерию max_features
        features = self.get_feature_ids(x.shape[1])
        impurity = np.zeros(x.shape[1])
        threshold = np.zeros(x.shape[1])
        
        for f_idx in features:
            imp, t = self.__find_threshold(x[:,f_idx], y)
            threshold[f_idx] = t
            impurity[f_idx] = imp
        
        # лучший признак по imputiry
        idx = np.argmin(impurity)
        
        x_left, x_right, y_left, y_right = self.__div_samples(x, y, idx, threshold[idx])
        
        if y_right.shape[0] == 0 or y_left.shape[0] == 0:
            #leaf
            elems_in_classes = np.bincount(y, minlength=2)
            best_class = np.argmax(elems_in_classes)
            probs = elems_in_classes/x.shape[0]
            self.tree[node_id] = (self.LEAF_TYPE, best_class, probs)
            return
        
        self.tree[node_id] = (self.NON_LEAF_TYPE, idx, threshold[idx])
        self.__fit_node(x_left, y_left, node_id*2 + 1, depth + 1)
        self.__fit_node(x_right, y_right, node_id*2 + 2, depth + 1)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [5]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [6]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.859153032303
1.93905591965


## Проверка качества работы

In [27]:
gkf = KFold(n_splits=5, shuffle=True)

In [32]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.932443668413
0.934356032261
0.932111083396
0.932027937141
0.930902590113


In [13]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.895485158394
0.89190986946
0.891702003825
0.890080651867
0.889909782564


Разница в скорости работы может объясняться универсальностью clf 

my_clf может использоваться лишь для бинарной классификации

In [58]:
print 'class 0 ', np.bincount(y)[0]
print 'class 1 ', np.bincount(y)[1]

class 0  111912
class 1  8357


Кроме того, виден явный дисбаланс в соотношении классов в данных